In [28]:
import numpy as np

def computeOPR(games):
    """
    Compute Offensive Power Rating (OPR) for FRC teams from match data.
    
    Parameters
    ----------
    games : list
        A list where each element is [blue_alliance, red_alliance, [blue_score, red_score]]
        Example:
        [
            [["254","1678","1323"], ["2056","1114","118"], [120, 115]],
            [["1678","2056","971"], ["254","1114","148"], [135, 125]]
        ]
    
    Returns
    -------
    dict
        Dictionary mapping each team -> OPR (float)
    """
    # 1. Collect all unique teams
    teams = sorted(set(sum([g[0] + g[1] for g in games], [])))
    team_index = {team: i for i, team in enumerate(teams)}
    n_teams = len(teams)
    n_matches = len(games) * 2  # each match contributes two alliance entries
    
    # 2. Build matrix A and score vector s
    A = np.zeros((n_matches, n_teams))
    s = np.zeros(n_matches)
    
    row = 0
    for blue, red, (blue_score, red_score) in games:
        for team in blue:
            A[row, team_index[team]] = 1
        s[row] = blue_score
        row += 1
        for team in red:
            A[row, team_index[team]] = 1
        s[row] = red_score
        row += 1
    
    # 3. Solve least squares A x = s
    x, _, _, _ = np.linalg.lstsq(A, s, rcond=None)
    
    # 4. Map results back to teams
    opr = {team: x[i] for team, i in team_index.items()}
    
    return opr


In [29]:
def computeWOPR(games, match_weights=None, ridge_alpha=0.0):
    """
    Compute weighted OPR from games.

    Parameters
    ----------
    games : list
        Each element: [blue_alliance, red_alliance, [blue_score, red_score]]
        team ids can be strings or ints.
    match_weights : None or list/array of length len(games)
        If None, all matches get equal weight 1. If provided, weight[i] is applied
        to both rows (blue and red) of games[i].
    ridge_alpha : float
        Regularization strength (0 => ordinary weighted least squares).

    Returns
    -------
    opr : dict team -> float
    team_index : dict team -> column index
    """
    # collect teams
    teams = sorted(set(sum([g[0] + g[1] for g in games], [])))
    team_index = {t: i for i, t in enumerate(teams)}
    n_teams = len(teams)
    n_rows = len(games) * 2

    # default weights
    if match_weights is None:
        match_weights = np.ones(len(games))
    else:
        match_weights = np.asarray(match_weights, dtype=float)
        if match_weights.shape[0] != len(games):
            raise ValueError("match_weights must have same length as games")

    # build A, s, and W (diagonal weights for rows)
    A = np.zeros((n_rows, n_teams))
    s = np.zeros(n_rows)
    W_diag = np.zeros(n_rows)

    r = 0
    for i, (blue, red, (blue_score, red_score)) in enumerate(games):
        w = float(match_weights[i])
        # blue row
        for t in blue:
            A[r, team_index[t]] = 1
        s[r] = blue_score
        W_diag[r] = w
        r += 1
        # red row
        for t in red:
            A[r, team_index[t]] = 1
        s[r] = red_score
        W_diag[r] = w
        r += 1

    # solve (A^T W A + alpha I) x = A^T W s
    # use W_diag to build A^T W A efficiently: multiply rows of A and s by sqrt(weight)
    sqrt_w = np.sqrt(W_diag)
    Aw = A * sqrt_w[:, None]     # each row scaled by sqrt(weight)
    sw = s * sqrt_w
    ATA = Aw.T @ Aw              # equivalent to A^T W A
    ATs = Aw.T @ sw              # equivalent to A^T W s

    if ridge_alpha and ridge_alpha > 0:
        ATA += ridge_alpha * np.eye(n_teams)

    x = np.linalg.solve(ATA, ATs)
    opr = {team: float(x[idx]) for team, idx in team_index.items()}
    
    return opr

In [30]:
games = [
    [["254","1678","1323"], ["2056","1114","118"], [120, 115]],
    [["1678","2056","971"], ["254","1114","148"], [135, 125]],
    [["1323","148","118"], ["971","254","1678"], [110, 140]]
]

opr = computeWOPR(games)
for team, score in sorted(opr.items(), key=lambda x: -x[1]):
    print(f"Team {team}: OPR = {score:.2f}")

opr['971'] + opr['254'] + opr['1678']

Team 1678: OPR = 125.00
Team 1114: OPR = 65.00
Team 148: OPR = 60.00
Team 118: OPR = 55.00
Team 971: OPR = 15.00
Team 254: OPR = -0.00
Team 2056: OPR = -5.00
Team 1323: OPR = -5.00


140.0

In [31]:
import requests

auth_key = 'utzEfVnhAbe9j7VlhxMggWg1XNvII1LZ6wBZm3mKwsBiBCfBimG8htWdQTAqIVU3'

def get_event_scores(event_key, auth_key):
    url = f"https://www.thebluealliance.com/api/v3/event/{event_key}/matches"
    headers = {"X-TBA-Auth-Key": auth_key}
    # url = f"https://theorangealliance.org/api/event/2526-CHN-HAQ/matches"
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

In [32]:
def totalScoreForGames(matches):
    games = []
    for m in matches:
        if m["comp_level"] != "qm":
            continue
        
        blue = [t[3:] for t in m["alliances"]["blue"]["team_keys"]]  # remove 'frc'
        red = [t[3:] for t in m["alliances"]["red"]["team_keys"]]
        blue_score = m["alliances"]["blue"]["score"] - m["score_breakdown"]["blue"]["foulPoints"]
        red_score = m["alliances"]["red"]["score"] - m["score_breakdown"]["blue"]["foulPoints"]
        if blue_score != -1 and red_score != -1:  # ignore unplayed matches
            games.append([blue, red, [blue_score, red_score]])
    return games

In [ ]:
def teleopScoreForGames(matches):
    games = []
    for m in matches:
        if m["comp_level"] != "qm":
            continue

        blue = [t[3:] for t in m["alliances"]["blue"]["team_keys"]]  # remove 'frc'
        red = [t[3:] for t in m["alliances"]["red"]["team_keys"]]
        blue_score = m["score_breakdown"]["blue"]["teleopCoralPoints"]
        red_score = m["score_breakdown"]["red"]["teleopCoralPoints"]
        if blue_score != -1 and red_score != -1:  # ignore unplayed matches
            games.append([blue, red, [blue_score, red_score]])
    return games

In [34]:
def autoScoreForGames(matches):
    games = []
    for m in matches:
        if m["comp_level"] != "qm":
            continue
        
        blue = [t[3:] for t in m["alliances"]["blue"]["team_keys"]]  # remove 'frc'
        red = [t[3:] for t in m["alliances"]["red"]["team_keys"]]
        blue_score = m["score_breakdown"]["blue"]["autoPoints"]
        red_score = m["score_breakdown"]["red"]["autoPoints"]
        if blue_score != -1 and red_score != -1:  # ignore unplayed matches
            games.append([blue, red, [blue_score, red_score]])
    return games

In [35]:
def getOPR(match, weighted): 
    matches = get_event_scores(match, auth_key)
    totalScores = totalScoreForGames(matches)
    teleopScores = teleopScoreForGames(matches)
    autoScores = autoScoreForGames(matches)
    #Unweighted
    if not weighted:
        totalOPR = computeOPR(totalScores)
        teleopOPR = computeOPR(teleopScores)
        autoOPR = computeOPR(autoScores)
        print('uw')

    #Weighted
    else:
        totalOPR = computeWOPR(totalScores)
        teleopOPR = computeWOPR(teleopScores)
        autoOPR = computeWOPR(autoScores)
        print('w')
    
    return totalOPR, teleopOPR, autoOPR

In [36]:
match = "2025cnsh"
totalOPR, teleopOPR, autoOPR = getOPR(match, True)

for team, score in sorted(totalOPR.items(), key=lambda x: -x[1]):
    totalScore = totalOPR[team]
    teleopScore = teleopOPR[team]
    autoScore = autoOPR[team]
    print(f"Team {team}: totalOPR = {totalScore}, teleopOPR = {teleopScore:.2f}, autoOPR = {autoScore:.2f}")

w
Team 6940: totalOPR = 61.276624716657906, teleopOPR = 32.55, autoOPR = 18.70
Team 7522: totalOPR = 60.901145846010316, teleopOPR = 44.53, autoOPR = 11.66
Team 4613: totalOPR = 57.79504101819911, teleopOPR = 25.24, autoOPR = 13.72
Team 6941: totalOPR = 57.34114110770417, teleopOPR = 35.19, autoOPR = 13.48
Team 5516: totalOPR = 56.82142719883907, teleopOPR = 30.09, autoOPR = 24.06
Team 5449: totalOPR = 55.35375945312836, teleopOPR = 34.89, autoOPR = 13.40
Team 8214: totalOPR = 55.21125176736576, teleopOPR = 26.51, autoOPR = 17.22
Team 9599: totalOPR = 48.089833462136745, teleopOPR = 22.79, autoOPR = 11.54
Team 8613: totalOPR = 43.665592681777674, teleopOPR = 29.63, autoOPR = 6.45
Team 10120: totalOPR = 40.545794458683936, teleopOPR = 27.31, autoOPR = 5.91
Team 10541: totalOPR = 39.73222156272471, teleopOPR = 18.26, autoOPR = 8.15
Team 10526: totalOPR = 36.899690036590634, teleopOPR = 19.34, autoOPR = 11.36
Team 6907: totalOPR = 36.83333320437205, teleopOPR = 27.68, autoOPR = 6.54
Team 